In [1]:
import pandas as pd

In [2]:
# Task 1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей,
# которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца
# timestamp для данного значения userId.

In [3]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
ratings_1 = ratings.groupby('userId').count()[['movieId']].sort_values('movieId', ascending = False).reset_index()
ratings_1.rename(columns={'movieId': 'number_of_ratings'}, inplace=True)
ratings_1.head()

,userId,number_of_ratings
0,547,2391
1,564,1868
2,624,1735
3,15,1700
4,73,1610


In [5]:
ratings_2 = ratings.groupby('userId').agg([min, max])['timestamp'].reset_index()
ratings_2['average_live_term'] = (ratings_2['max'] - ratings_2['min'])/3600/24
ratings_2.head()

,userId,min,max,average_live_term
0,1,1260759108,1260759205,0.001123
1,2,835355395,835356246,0.009850
2,3,1298861589,1298932787,0.824051
3,4,949778714,949982274,2.356019
4,5,1163373044,1163375145,0.024317


In [6]:
result = ratings_1.merge(ratings_2, on = 'userId', how = 'left')
result

,userId,number_of_ratings,min,max,average_live_term
0,547,2391,974777109,1476587644,5807.992303
1,564,1868,974708446,976224804,17.550440
2,624,1735,1019123866,1476616373,5295.052164
3,15,1700,997937239,1469330735,5455.943241
4,73,1610,1255500852,1476086345,2553.072836
...,...,...,...,...,...
666,296,20,1298158632,1298166310,0.088866
667,289,20,1328935420,1328935859,0.005081
668,249,20,839461218,839463782,0.029676
669,221,20,1111551492,1111551662,0.001968


In [9]:
result.query('number_of_ratings > 100', inplace=True)
result.sort_values('average_live_term', ascending = False)

,userId,number_of_ratings,min,max,average_live_term
7,380,1063,949367125,1465156786,5969.787743
0,547,2391,974777109,1476587644,5807.992303
83,427,294,939332587,1415901843,5515.847870
3,15,1700,997937239,1469330735,5455.943241
2,624,1735,1019123866,1476616373,5295.052164
...,...,...,...,...,...
225,110,120,840100360,840101508,0.013287
253,217,104,1108160000,1108160981,0.011354
190,316,140,1460822646,1460823562,0.010602
191,609,140,1029869519,1029870367,0.009815


In [11]:
result['average_live_term'].mean()/365 #потратить больше года на оценку фильмов еще куда не шло

1.270944553830936

In [18]:
result['average_live_term'][7]/365 #но 16 лет? невольно возникает вопрос о корректности расчетов :) но, ошибку не нашла

16.355582857686453

In [ ]:
# Task 2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия).
# Необходимо сформировать две таблицы:
# таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
# аналогичную таблицу по типам выручки с указанием адреса клиента
# Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

In [ ]:
total_revenue = rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')
total_revenue

In [ ]:
total_rev_with_adress = client_base.merge(total_revenue.fillna(0), on='client_id', how='left')
total_rev_with_adress